In [110]:
import os

In [111]:
%pwd

'c:\\Users\\jaide\\OneDrive\\Desktop\\GOPAL\\Projects\\Machine Learning Project\\machine-learning-project'

In [112]:
# os.chdir("../")

In [113]:
%pwd

'c:\\Users\\jaide\\OneDrive\\Desktop\\GOPAL\\Projects\\Machine Learning Project\\machine-learning-project'

In [114]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str


In [115]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml , create_directories , save_json

In [116]:
class ConfigurationManager:
    def __init__ (
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
             root_dir=config.root_dir,
             test_data_path= config.test_data_path,
             model_path= config.model_path,
             all_params= params,
             metric_file_name= config.metric_file_name,
             target_column= schema.name,
        )

        return model_evaluation_config

In [117]:
import os
import pandas as pd 
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# from urlib.parse import urlparse
import numpy as np
import joblib

In [118]:
class ModelEvaluation:
   def __init__(self, config: ModelEvaluationConfig):
        self.config = config

   def eval_metrics(self,actual, pred):
     rmse = np.sqrt(mean_squared_error(actual,pred))
     mae =  mean_absolute_error(actual, pred)
     r2 = r2_score(actual, pred)

     return rmse, mae, r2
    
   def save_results(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[self.config.target_column]

        predicted_qualities = model.predict(test_x)

        (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)


        scores = {"rmse": rmse, "mae":mae, "r2":r2}
        save_json(Path(self.config.metric_file_name) ,scores)

In [119]:
try:
    config=ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config= ModelEvaluation(config= model_evaluation_config)
    model_evaluation_config.save_results()
    
except Exception as e:
    raise e

[2025-05-23 16:11:17,925: INFO: common: yaml file: C:\Users\jaide\OneDrive\Desktop\GOPAL\Projects\Machine Learning Project\machine-learning-project\config\config.yaml loaded successfully]
[2025-05-23 16:11:17,931: INFO: common: yaml file: C:\Users\jaide\OneDrive\Desktop\GOPAL\Projects\Machine Learning Project\machine-learning-project\params.yaml loaded successfully]
[2025-05-23 16:11:17,937: INFO: common: yaml file: C:\Users\jaide\OneDrive\Desktop\GOPAL\Projects\Machine Learning Project\machine-learning-project\schema.yaml loaded successfully]
[2025-05-23 16:11:17,940: INFO: common: created directory at: artifacts]
[2025-05-23 16:11:17,946: INFO: common: created directory at: artifacts/model_evaluation]
[2025-05-23 16:11:17,966: INFO: common: json file saved at : artifacts\model_evaluation\metrics.json]
